In [174]:
!ls

'Colab Notebooks'   Housing.csv   train.csv


In [175]:
%cd drive/MyDrive

[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive


In [176]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
df = pd.read_csv("train.csv")

In [177]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


ПРЕДОБРАБОТКА ДАННЫХ.

По первым 5 строкам нашего датафрейма и для построения модели для определения класса Survived - следует удалить столбцы "PassengerId" и "Name" так как для предсказания "Выжил - Не выжил" они не несут в себе какую-либо значимость. "Пол" следует переопределить в категории. Номер билета и его стоимость (Fare) также можно исключить из выборки. "Pclass", "Кабины" и "Место посадки" (Embarked) следует переопределить в категории.
Значения количества родственников (SibSp) и количества родителей, детей (Parch) предлагаю оставить как есть, так как это не качественный (категориальный), а количественный признак.

In [178]:
df.pop("PassengerId")
df.pop("Name")
df.pop("Ticket")
df.pop("Fare")

0       7.2500
1      71.2833
2       7.9250
3      53.1000
4       8.0500
        ...   
886    13.0000
887    30.0000
888    23.4500
889    30.0000
890     7.7500
Name: Fare, Length: 891, dtype: float64

In [179]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Cabin,Embarked
0,0,3,male,22.0,1,0,NaN,S
1,1,1,female,38.0,1,0,C85,C
2,1,3,female,26.0,0,0,NaN,S
3,1,1,female,35.0,1,0,C123,S
4,0,3,male,35.0,0,0,NaN,S


С помощью OneHotEncoder преобразуем данные столбца 'Sex' в категориальные данные

In [180]:
ohe = OneHotEncoder()
res = ohe.fit_transform(df['Sex'].to_numpy().reshape(-1, 1))
ohe_df = pd.DataFrame(res.toarray(), columns = ohe.get_feature_names_out())
ohe_df.head()

,x0_female,x0_male
0,0.0,1.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [181]:
df = pd.concat([df, ohe_df], axis=1).drop(columns='Sex')

In [182]:
df.head()

,Survived,Pclass,Age,SibSp,Parch,Cabin,Embarked,x0_female,x0_male
0,0,3,22.0,1,0,NaN,S,0.0,1.0
1,1,1,38.0,1,0,C85,C,1.0,0.0
2,1,3,26.0,0,0,NaN,S,1.0,0.0
3,1,1,35.0,1,0,C123,S,1.0,0.0
4,0,3,35.0,0,0,NaN,S,0.0,1.0


С помощью OneHotEncoder преобразуем данные столбца 'Pclass' в категориальные данные

In [183]:
ohe2 = OneHotEncoder(sparse_output = False)
res2 = pd.DataFrame(ohe2.fit_transform(df[['Pclass']]))
res2.head()

,0,1,2
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [184]:
ohe2.get_feature_names_out()

array(['Pclass_1', 'Pclass_2', 'Pclass_3'], dtype=object)

In [185]:
res2.columns = ohe2.get_feature_names_out()
res2.head()

,Pclass_1,Pclass_2,Pclass_3
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [186]:
df = pd.concat([df, res2], axis=1).drop(columns='Pclass')
df.head()

,Survived,Age,SibSp,Parch,Cabin,Embarked,x0_female,x0_male,Pclass_1,Pclass_2,Pclass_3
0,0,22.0,1,0,NaN,S,0.0,1.0,0.0,0.0,1.0
1,1,38.0,1,0,C85,C,1.0,0.0,1.0,0.0,0.0
2,1,26.0,0,0,NaN,S,1.0,0.0,0.0,0.0,1.0
3,1,35.0,1,0,C123,S,1.0,0.0,1.0,0.0,0.0
4,0,35.0,0,0,NaN,S,0.0,1.0,0.0,0.0,1.0


С помощью OneHotEncoder преобразуем данные столбца 'Cabin' в категориальные данные

In [187]:
ohe3 = OneHotEncoder(sparse_output = False)
res3 = pd.DataFrame(ohe3.fit_transform(df[['Cabin']]))
res3.head()

,0,1,2,3,4,5,6,7,8,9,...,138,139,140,141,142,143,144,145,146,147
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [188]:
res3.columns = ohe3.get_feature_names_out()
res3.head()

,Cabin_A10,Cabin_A14,Cabin_A16,Cabin_A19,Cabin_A20,Cabin_A23,Cabin_A24,Cabin_A26,Cabin_A31,Cabin_A32,...,Cabin_F E69,Cabin_F G63,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [189]:
df = pd.concat([df, res3], axis=1).drop(columns='Cabin')
df.head()

,Survived,Age,SibSp,Parch,Embarked,x0_female,x0_male,Pclass_1,Pclass_2,Pclass_3,...,Cabin_F E69,Cabin_F G63,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_nan
0,0,22.0,1,0,S,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,38.0,1,0,C,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,26.0,0,0,S,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,35.0,1,0,S,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,35.0,0,0,S,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


С помощью OneHotEncoder преобразуем данные столбца 'Embarked' в категориальные данные

In [190]:
ohe4 = OneHotEncoder(sparse_output = False)
res4 = pd.DataFrame(ohe4.fit_transform(df[['Embarked']]))
res4.head()

,0,1,2,3
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [191]:
res4.columns = ohe4.get_feature_names_out()
res4.head()

,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [192]:
df = pd.concat([df, res4], axis=1).drop(columns='Embarked')
df.head()

,Survived,Age,SibSp,Parch,x0_female,x0_male,Pclass_1,Pclass_2,Pclass_3,Cabin_A10,...,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,0,22.0,1,0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1,38.0,1,0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,26.0,0,0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1,35.0,1,0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,35.0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [193]:
df.isna().sum().sort_values()

Survived           0
Cabin_D15          0
Cabin_D17          0
Cabin_D19          0
Cabin_D20          0
                ... 
Cabin_B96 B98      0
Cabin_C101         0
Cabin_B78          0
Embarked_nan       0
Age              177
Length: 161, dtype: int64

177 значений в столбце Age пропущены, считаю, что для корректного построения модели - следует исключить 177 строк. Можно конечно присвоить какие-либо данные в эту выборку, но это будет искажать корректный, фактический результат.

In [194]:
df.dropna(inplace=True)

In [195]:
df.isna().sum().sort_values()

Survived         0
Cabin_D15        0
Cabin_D17        0
Cabin_D19        0
Cabin_D20        0
                ..
Cabin_B94        0
Cabin_B96 B98    0
Cabin_C101       0
Cabin_C50        0
Embarked_nan     0
Length: 161, dtype: int64

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Columns: 161 entries, Survived to Embarked_nan
dtypes: float64(158), int64(3)
memory usage: 903.7 KB


In [197]:
data = df.drop(columns='Survived')
data.head()

,Age,SibSp,Parch,x0_female,x0_male,Pclass_1,Pclass_2,Pclass_3,Cabin_A10,Cabin_A14,...,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Cabin_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,22.0,1,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,38.0,1,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,26.0,0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,35.0,1,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,35.0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [198]:
y = df["Survived"]
x = data

Разделим наши данные на тестовую и тренировочную выборки с фиксированием random_state для воспроизводимости процесса.

In [199]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2024, test_size=0.25)

Cледует масштабировать наши данные, для корректного построения модели

In [200]:
scaler = StandardScaler()
x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
x_test_scaled = pd.DataFrame(scaler.fit_transform(x_test), columns=x_test.columns)

Измерение качества на тестовой выборке.

1) Используем dummyclassifier для получения (и понимания) самой минимальной возможной точности нашей модели.
Другими словами - измерение качества константного предсказания

In [201]:
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(x_train_scaled, y_train)
y_pred_dummy = dummy_clf.predict(x_test_scaled)
accuracy_score(y_test, y_pred_dummy)

0.6089385474860335

2) Используем модель логистической регрессии

In [202]:
log_reg = LogisticRegression()
log_reg.fit(x_train_scaled, y_train)
y_pred_log_reg = log_reg.predict(x_test_scaled)
accuracy_score(y_test, y_pred_log_reg)

0.7877094972067039

Результат - 79% точность (Accuracy) предсказания на тестовой выборке. Достаточно хороший результат, но учитывая, что, все-таки, наши классы Выжил - Не выжил - неравные, так как, в общем, мы знаем что выживших в титанике гораздо меньше чем невыживших, то использование данной метрики *Нецелесообразно*

Метрики Precision и Recall подходят для нашего задания, в случае несбалансированных классов. Также можно использовать F-меру - как некоторый средний показатель по двум критериям Precision и Recall.
Результаты представлены ниже.

In [203]:
confusion_matrix(y_test, y_pred_dummy)

array([[109,   0],
       [ 70,   0]])

In [204]:
confusion_matrix(y_test, y_pred_log_reg)

array([[89, 20],
       [18, 52]])

In [205]:
print(f'Precision for DummyClassificator: {precision_score(y_test, y_pred_dummy, zero_division=0)}')
print(f'Precision for Logistic regression: {precision_score(y_test, y_pred_log_reg, zero_division=0)}')

Precision for DummyClassificator: 0.0
Precision for Logistic regression: 0.7222222222222222


In [206]:
print(f'Recall for DummyClassificator: {recall_score(y_test, y_pred_dummy, zero_division=0)}')
print(f'Recall for Logistic regression: {recall_score(y_test, y_pred_log_reg, zero_division=0)}')

Recall for DummyClassificator: 0.0
Recall for Logistic regression: 0.7428571428571429


In [207]:
print(f'F1-score for DummyClassificator: {f1_score(y_test, y_pred_dummy)}')
print(f'F1-score for Logistic regression: {f1_score(y_test, y_pred_log_reg)}')

F1-score for DummyClassificator: 0.0
F1-score for Logistic regression: 0.732394366197183


В целом по метрике F1-score точность нашей модели составляет 73%, что довольно неплохо, учитывая сложность предсказания "Выжил - Не выжил" с учетом мало значимых имеющихся данных (Например Пол, Место посадки пассажира (Embarked)). Возможно через кросс-валидацию мы сможем переобучить нашу модель и немного повысить точность, подстроив ее максимально под имеющиеся данные, но скорее всего на абсолютно новых данных она будет работать некорректно.  